# Jupyter Notebooks on Tetralith

Jupyter Notebooks have become a mainstream tool for interactive computing in every field of science. Jupyter is slowly making its way into HPC, and NSC now offers access to Jupyter on the Tetralith cluster.

## Contents

- [Starting Jupyter under ThinLinc](#Starting-Jupyter-under-ThinLinc-at-NSC)
 - [Running a notebook on a login node](#Running-a-notebook-on-a-login-node)
 - [Running a notebook on a compute node](#Running-a-notebook-on-a-compute-node)
- [Starting Jupyter using an ssh tunnel](#Starting-Jupyter-using-an-ssh-tunnel)
 - [Running a notebook on a login node](#Running-a-notebook-on-a-login-node)
- [Optional: Jupyter configuration](#Jupyter-configuration)
- [Optional: Setting a Jupyter password](#Optional:-Setting-a-Jupyter-password)
- [Optional: Generate a SSL certificate](#Optional:-Generate-a-SSL-certificate)
- [Fetch these sources](#Fetch-these-sources)

## Starting Jupyter under ThinLinc at NSC

Starting Jupyter under ThinLinc is done in a few simple steps.

### Running a notebook on a login node

1. Start a terminal
2. `tetralith~$ module load Python/3.6.3-anaconda-5.0.1-nsc1`
3. `tetralith~$ jupyter-notebook`

**Note**: only lightweight tasks should be performed on the login node, such as light visualization work and submitting jobs to the SLURM scheduler.

### Running a notebook on a compute node

1. Start a terminal
2. `tetralith~$ interactive -N 1 -t 120 -A snic2018-x-yy`
3. `n1~$ module load Python/3.6.3-anaconda-5.0.1-nsc1`
4. `n1~$ jupyter-notebook --no-browser --ip=n1`
5. `tetralith~$ firefox`
6. Copy the URL from the interactive terminal on n1 and paste it into firefox (not starting Firefox first, but just pressing Ctrl and clicking on the URL may also work)

**Note**: The node name `n1` is just an example name. You should use the name of the node you get allocated for the `--ip` option.

## Starting Jupyter using an ssh tunnel

If you would rather use a browser on your local laptop/desktop, you can instead use an ssh tunnel. *However, this currently only works for running notebooks on the login nodes.*

### Running a notebook on a login node

1. `mylaptop~$ ssh -L 9988:localhost:9988 x_abcde@tetralith.nsc.liu.se`
2. `tetralith~$ module load Python/3.6.3-anaconda-5.0.1-nsc1`
3. `tetralith~$ jupyter-notebook --port=9988 --no-browser`

Then copy-and-paste the URL displayed in the terminal into a browser running on "mylaptop".

**Note1**: The port 9988 is somewhat arbitrary but less risk to collide with other users than the default 8888. If 9988 is already in use, then just try 9989, etc.

**Note2**: Picking the same (laptop) local port as on the login node allows the URL to be directly copy-and-pasted, but this is of course not strictly necessary.

**Note3**: Only lightweight tasks should be performed on the login node, such as visualization work and submitting jobs to the SLURM scheduler.

## Fetch these sources

Log in to Tetralith, clone the repository with the lesson material and checkout the `tetralith` branch:

```bash
$ ssh <username>@tetralith.nsc.liu.se
$ git clone https://github.com/PDC-support/jupyter-notebook.git
$ git checkout tetralith
```

## Optional: Jupyter configuration

Various configuration options for Jupyter can be set by creating a config file:
- Load the appropriate module:

```bash
$ module load Python/3.6.3-anaconda-5.0.1-nsc1
```

- Generate the configuration file `$HOME/.jupyter/jupyter_notebook_config.py`: 

```bash
$ jupyter notebook --generate-config
```

The configuration file will have many lines with possible configuration options, but all of them will be commented out initially.

In order to start IPython clusters and use `ipyparallel` to run parallel tasks in a Jupyter notebook, one also needs to generate an IPython config file:

- Generate the configuration file `$HOME/.ipython/profile_default/ipython_config.py`: 

```bash
$ ipython profile create
```



## Optional: Setting a Jupyter password

- Create your Jupyter password by: 

```
$ jupyter notebook password
```

- The entered password will be saved in hashed form in the file `$HOME/.jupyter/jupyter_notebook_config.json`.

## Optional: Generate a SSL certificate

To enable higher security using the https protocol, you need to generate an SSL certificate.

- Generate a self-signed certificate with openssl in a private directory:

```bash
$ cd $HOME/Private # some private folder
$ openssl req -x509 -nodes -days 30 -newkey rsa:2048 -keyout mykey.key -out mycert.pem
```

- Add information on the certificate in your Jupyter config file $HOME/.jupyter/jupyter_notebook_config.py:

```python
c.NotebookApp.certfile = u'/home/x_abcde/Private/mycert.pem'
c.NotebookApp.keyfile = u'/home/x_abcde/Private/mykey.key'
```

- After these steps, the URL that shows up in the output of the jupyter-notebook command should be https. Your browser will likely complain that your self-signed certificate is not secure or not recognized (since it is self-signed). Tell it to proceed anyways.

## Fetch these sources

Log in to Tetralith, clone the repository with the lesson material and checkout the `tetralith` branch:

```bash
$ ssh <username>@tetralith.nsc.liu.se
$ git clone https://github.com/PDC-support/jupyter-notebook.git
$ git checkout tetralith
```